In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, hamming_loss
from tqdm import tqdm

c:\Users\Anas\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# 1. Load Dataset
df = pd.read_csv("path of dataset ")
label_columns = [col for col in df.columns if col not in ['text', 'id']]
df['label_list'] = df[label_columns].apply(lambda row: [label_columns[i] for i, val in enumerate(row) if val == 1], axis=1)


In [7]:

# 2. Binarize Labels
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['label_list'])

In [8]:
# 3. Split Data
X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.2, random_state=42)


In [9]:
# 4. Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
class EmotionDataset(Dataset):
    def __init__(self, texts, labels):
        self.encodings = tokenizer(list(texts), truncation=True, padding=True, max_length=128)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

In [11]:
train_dataset = EmotionDataset(X_train, y_train)
test_dataset = EmotionDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8)

In [12]:

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(label_columns),
    problem_type="multi_label_classification"
)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:

# 6. Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

In [15]:
# 1. Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Move the model to the device
model.to(device)



DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [18]:
# Training loop
model.train()
for epoch in range(1):
    total_loss = 0
    for batch in tqdm(train_loader):
        # Remove token_type_ids for DistilBERT
        batch = {k: v.to(device) for k, v in batch.items() if k != 'token_type_ids'}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}")


100%|██████████| 21123/21123 [34:45<00:00, 10.13it/s]

Epoch 1, Loss: 0.1177


In [23]:
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        labels = batch['labels']
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels' and k != 'token_type_ids'}
        outputs = model(**inputs)
        preds = torch.sigmoid(outputs.logits).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())


In [24]:
import numpy as np


In [31]:
# Apply threshold for multi-label
threshold = 0.3
y_pred = np.array(all_preds) >= threshold
y_true = np.array(all_labels)



In [32]:
f1 = f1_score(y_true, y_pred, average="macro")
hl = hamming_loss(y_true, y_pred)

In [33]:
print(f"F1 Score: {f1:.4f}")
print(f"Hamming Loss: {hl:.4f}")

F1 Score: 0.3144
Hamming Loss: 0.0419


In [34]:
# 9. Example Inference
def predict_emotions(text):
    tokens = tokenizer([text], padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)
    output = model(**tokens)
    probs = torch.sigmoid(output.logits).detach().cpu().numpy()[0]
    predicted_labels = [mlb.classes_[i] for i, prob in enumerate(probs) if prob >= threshold]
    return predicted_labels

In [39]:
example = "I am not happy with the service"
print("Predicted Emotions:", predict_emotions(example))

Predicted Emotions: ['disapproval']


In [1]:
from ipywidgets import Widget
Widget.widget_types = {}
